<a href="https://colab.research.google.com/github/AbrahamAzizi/ComputerVision/blob/main/ImageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import os
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')
gdrive_cwd = '/content/gdrive/My Drive/Colab Notebooks'
dataset_dir = os.path.join(gdrive_cwd, 'Classification_Dataset')

Mounted at /content/gdrive


In [ ]:
img_h = 32
img_w = 32
bs = 8
nb_epoch = 50
verbose = 1
validation_split = 0.01
SEED = 8585
optim = RMSprop()
num_classes = 20
classes_list = [
               'owl',               # 0
               'galaxy',            # 1
               'lightning',         # 2
               'wine-bottle',       # 3
               't-shirt',           # 4
               'waterfall',         # 5
               'sword',             # 6
               'school-bus',        # 7
               'calculator',        # 8
               'sheet-music',       # 9
               'airplanes',         # 10
               'lightbulb',         # 11
               'skyscraper',        # 12
               'mountain-bike',     # 13
               'fireworks',         # 14
               'computer-monitor',  # 15
               'bear',              # 16
               'grand-piano',       # 17
               'kangaroo',          # 18
               'laptop']           # 19
color_mode = 'rgb' 
img_ch = 3

In [ ]:
# ImageDataGenerator
image_data_generator = ImageDataGenerator(rotation_range=40,
                                          width_shift_range=.2,
                                          height_shift_range=.2,
                                          zoom_range=0.3,
                                          horizontal_flip=True,
                                          vertical_flip=True,                                         
                                          fill_mode='constant',
                                          cval=0,
                                          rescale=1./255,
                                          validation_split=validation_split)

# Training Data Generator
training_dir = os.path.join(dataset_dir, 'training')
train_gen = image_data_generator.flow_from_directory(training_dir,
                                               batch_size=bs,
                                               color_mode=color_mode,
                                               class_mode='categorical',
                                               classes=classes_list,
                                               shuffle=True,
                                               subset='training',
                                               seed=SEED,
                                               target_size=(img_h, img_w))  # targets are directly converted into one-hot vectors


valid_gen = image_data_generator.flow_from_directory(training_dir,
                                               batch_size=bs,
                                               color_mode=color_mode, #rgb,grayscale
                                               class_mode='categorical',
                                               classes=classes_list,
                                               shuffle=False,
                                               subset='validation',
                                               seed=SEED,
                                               target_size=(img_h, img_w))

Found 1522 images belonging to 20 classes.
Found 3 images belonging to 20 classes.


In [ ]:
# Training data set
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, img_ch], [None, num_classes]))
print(train_dataset)
train_dataset = train_dataset.repeat()

# Validation
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, img_ch], [None, num_classes]))
print(valid_dataset)
# Repeat
valid_dataset = valid_dataset.repeat()

#test data set
test_data_gen = ImageDataGenerator(rescale=1./255)
test_dir = os.path.join(dataset_dir, 'test')
test_gen = test_data_gen.flow_from_directory(test_dir,
                                             batch_size=1,
                                             classes=classes_list,
                                             color_mode=color_mode,
                                             class_mode='categorical',
                                             shuffle=False,
                                             target_size=(img_h, img_w),
                                             seed=SEED)


test_dataset = tf.data.Dataset.from_generator(lambda: test_gen,
                                              output_types=(tf.float32, tf.float32),
                                              output_shapes=([None, img_h, img_w, img_ch], [None, num_classes]))

print(test_dataset)

<FlatMapDataset element_spec=(TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 20), dtype=tf.float32, name=None))>
<FlatMapDataset element_spec=(TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 20), dtype=tf.float32, name=None))>
Found 500 images belonging to 20 classes.
<FlatMapDataset element_spec=(TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 20), dtype=tf.float32, name=None))>


### Model architecture should be designed and implemented in this block.

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same',
input_shape=(img_h, img_w, img_ch)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        1792      
                                                                 
 activation (Activation)     (None, 32, 32, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        36928     
                                                                 
 activation_1 (Activation)   (None, 32, 32, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 64)        0         
                                                        

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = optim , metrics = ['accuracy'])

In [ ]:
model.fit(x=train_dataset,
          epochs=5,  
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen),
          callbacks=None)

Epoch 1/5
191/191 [==============================] - 1196s 6s/step - loss: 2.9832 - accuracy: 0.0690 - val_loss: 2.5778 - val_accuracy: 0.3333
Epoch 2/5
191/191 [==============================] - 9s 45ms/step - loss: 2.8839 - accuracy: 0.0992 - val_loss: 2.1767 - val_accuracy: 0.3333
Epoch 3/5
191/191 [==============================] - 9s 45ms/step - loss: 2.7790 - accuracy: 0.1386 - val_loss: 2.4086 - val_accuracy: 0.0000e+00
Epoch 4/5
191/191 [==============================] - 9s 45ms/step - loss: 2.7219 - accuracy: 0.1452 - val_loss: 2.2645 - val_accuracy: 0.3333
Epoch 5/5
191/191 [==============================] - 9s 45ms/step - loss: 2.6618 - accuracy: 0.1662 - val_loss: 2.5244 - val_accuracy: 0.0000e+00


In [ ]:
test_file_names = test_gen.filenames
predictions = model.predict_generator(test_dataset, len(test_file_names))